# VSCode Jupyter Remote Server
This notebook starts a private remote Jupyter notebook sever that you can connect to using VSCode's remote kernel mode.

## Overview
This notebook downloads and runs [NGrok](https://ngrok.com) to create a reverse tunnel to the Jupyter notebook server running 'locally' on Kaggle.

Because this method bypasses Jupyter's authentication, use protect the NGrok tunnel with a username and password.

## Setup

### NGrok
This notebook using [NGrok](https://ngrok.com/) to create a 'reverse tunnel'. Sign up for a free account, and make a note of your `auth token`

### Secrets
If running this on [Kaggle](https://www.kaggle.com/), this notebook uses 'Kaggle Secrets' so your auth token isn't stored in the notebook or viewable by others.
On the Kaggle notebook editor page, click on 'Add-Ons', then 'Secrets'. Create 3 secrets:
 * `NGROK-AUTH-TOKEN` - Your Auth token from the NGrok dashboard
 * `NGROK-USERNAME` - The username for the tunnel's HTTP authentication
 * `NGROK-PASSWORD` - The username for the tunnel's HTTP authentication
 
 If not on Kaggle, the code will look for those values in the program's Environment Variables.
 
 To run the tunnel without a username or password, set the `NGROK-UNSAFE-NO-PASSWORD` secret/env variable.

  **-------------------------------**
  
  **NOTE** running without a username and password is unsafe - NGrok URLs are only obfuscated, mean they can discovered by strangers who can then
 connect into your notebook environment and **RUN ARBITRARY CODE** inside it. Only use this when you really know what you're doing
 
  **-------------------------------**

In [1]:
#hide
# Helper functions
import os
import subprocess

# Check if we are running on Kaggle or not
try:
    from kaggle_secrets import UserSecretsClient, BackendError
    user_secrets = UserSecretsClient()
    on_kaggle = True
except ImportError:
    on_kaggle = False

# Run a process and optionally get the output
def run(cmd, return_output=False):
    p = subprocess.run(cmd, shell=True, capture_output=True, check=True)
    sout = p.stdout.decode().strip()
    serr = p.stderr.decode().strip()
    ret = "\n".join([sout, serr])
    if return_output:
        return ret
    print(ret)
    return None

# Look for a variable either in Kaggle secrets on the programs'
# environment variables
def get_variable(name, allow_missing=False):
    if on_kaggle:
        if allow_missing:
            try:
                return user_secrets.get_secret(name)
            except BackendError:
                return None
        else:
            return user_secrets.get_secret(name)
    else:
        if allow_missing:
            return os.environ.get(name, None)
        else:
            return os.environ[name]

In [ ]:
# Install Ngrok
cmd = """
curl -o ngrok.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz &&
tar zxvf ngrok.tgz &&
chmod ugo+rwx ngrok &&
mv ngrok /usr/bin
""".replace("\n", " ")
run(cmd)

In [ ]:
# Add auth token    
auth_token = get_variable("NGROK-AUTH-TOKEN")
run(f'ngrok config add-authtoken "{auth_token}"')

In [ ]:
# Get URL path to add to NGROK host
import json
j = json.loads(run("jupyter notebook list --json", return_output=True))
url_path = j["base_url"]

In [ ]:
# Start tunnel
skip_password = get_variable("NGROK-UNSAFE-NO-PASSWORD", allow_missing=True)
if skip_password is None:
    username = get_variable("NGROK-USERNAME")
    password = get_variable("NGROK-PASSWORD")
    cmd = f'ngrok http --basic-auth "{username}:{password}" --log stdout --log-format json 8888'
else:
    print("[**] NOTE, not using password to secure endpoint, this is unsafe!")
    cmd = f'ngrok http --log stdout --log-format json 8888'

p = subprocess.Popen([cmd], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

In [ ]:
# Read process output until we find the URL
while True:
    j = json.loads(p.stdout.readline().decode())
    print(json.dumps(j))
    if "url" in j:
        base_url = j["url"]
        break

In [ ]:
# Print URL to connect to
url = base_url + url_path
print(f"[*] Connect to: {url}")

In [ ]:
# Wait 'forever' until process has finished
p.wait()